In [3]:
include("../models/ReusableResourceAllocation.jl")
using .ReusableResourceAllocation: reusable_resource_allocation
function contagion_model( 
    adj_matrix, 
    contaminated_hospitals, 
    initial_capacities, 
    patients,
    T::Real=0
)
    #= 
        adj_matrix [N by N matrix]: specifies the edges between hospitals in a network; N is the number 
                    of hospitals in the network

        contaminated hospitals [array of size N]: an array where index i is 1 if the hospital is contaminated 
                                0 otherwise

        initial_capacities [array of size N]: an array where index i is the number of initial capacities in hospital i 
                            
        patients [N by T matrix]: each entry [i][t] tells us the amount of new patients we see at hospital i at time t

        T [int]: the time horizon
    =#

    # used to create new graph of the exact same size 
    N = length(contaminated_hospitals)

    # construct the Stage 1 graph 

    stage1_matrix = falses(N,N)
    
    for i in 1:N  
        for j in 1:N 
            # check that the edge is directed toward the contaminated hospital
            # if true set the matrix idx to 1 else leave it as 0 
            stage1_matrix[i,j] = (contaminated_hospitals[j] == 1)
        end
    end

    # construct the Stage 2 graph 
    stage2_matrix = falses(N,N)
    for i in 1:N  
        for j in 1:N 
            # Manditorily make a fully connected graph among non-contaminated hospitals
            stage2_matrix = ((contaminated_hospitals[i] == 0) && contaminated_hospitals[j] == 0)
        end
    end

    for t in 1:T
        ###########
        # STAGE 1 #
        ###########
        # In stage 1 we only run resource redistribution model on a modified network.
        # The modified network is as follows: the only edges that exist are those that are directed to the contaminated
        # hosptials. In other words, we can only send resources to the contaminated hospitals in this stage. 
        # To accomplish this, we run a resource redistribtuion model. 

        reusable_resource_allocation(
            initial_supply,
            supply,
            demand,
            adj_matrix,
            obj_dir,
            send_new_only,
            sendrecieve_switch_time,
            min_send_amt,
            smoothness_penalty,
            setup_cost,
            sent_penalty,
            verbose,
        )
    end 
end

contagion_model (generic function with 2 methods)

In [23]:
import XLSX
xf = XLSX.readxlsx("CaseCountData.xlsx")
sheetnames = XLSX.sheetnames(xf)
xf["Trends"]

9-element Vector{String}:
 "Case and Fatalities_ALL"
 "Case and Fatalities_2020"
 "Case and Fatalities_2021"
 "Case and Fatalities_2022"
 "Case and Fatalities_2023"
 "Trends"
 "Molecular Positivity Rate"
 "Hospitalizations"
 "Hospitalization by Day"

In [26]:
using DataFrames, XLSX
df = DataFrame(XLSX.readtable("CaseCountData.xlsx", "Trends"))

Row,Date,Cumulative Confirmed Cases,New Confirmed Cases,Cumulative Probable Cases,New Probable Cases,Cumulative Fatalities,Total Fatalities Current Day
,Any,Any,Any,Any,Any,Any,Any
1,2020/03/04,0,0,0,0,missing,missing
2,2020/03/05,0,0,0,0,missing,missing
3,2020/03/06,1,1,0,0,missing,missing
4,2020/03/07,1,0,0,0,0,missing
5,2020/03/08,3,2,1,1,0,missing
6,2020/03/09,9,6,1,0,0,missing
7,2020/03/10,19,10,2,1,0,0
8,2020/03/11,25,6,3,1,0,0
9,2020/03/12,30,5,3,0,0,0
